In [12]:
#All imports go here
from keras.models import Sequential
from keras.layers import Convolution2D, Flatten,Dense, Lambda, MaxPooling2D

import cv2
import numpy as np
import pandas as pd

In [2]:
# Load data
DATA_DIR = './data/'
LOGFILE_PATH = DATA_DIR + 'driving_log.csv'
IMG_DIR = DATA_DIR + 'IMG/'

logs = pd.read_csv(LOGFILE_PATH,sep=',',header=None)


In [36]:
# Load image data.
img_paths = {}
img_paths['center'] = list(logs[0])

image_data = []

for path in img_paths['center']:
    file_name = path.split('/')[-1]
    img = cv2.imread(IMG_DIR+file_name)
    image_data.append(img)

image_data = np.array(image_data)


#Load steering angles
steering_angles  = np.array(list(logs[3]))

In [37]:
# Augment data by flipping images and steering angles.
image_data_flipped = np.empty_like(image_data)

for i in range(len(image_data)):
    image_data_flipped[i] = np.fliplr(image_data[i])
    
steering_angles_flipped = -steering_angles


image_data = np.vstack((image_data,image_data))
steering_angles = np.hstack((steering_angles,steering_angles_flipped))


In [42]:
# Build and save model.
model = Sequential()
model.add(Lambda(lambda x: x/255 - 0.5,input_shape=image_data.shape[1:]))
model.add(Convolution2D(6,5,5,activation='relu'))
model.add(MaxPooling2D())
model.add(Convolution2D(16,5,5,activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120,activation='relu'))
model.add(Dense(84,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.fit(image_data, steering_angles,  nb_epoch=1, validation_split=0.2, shuffle=True,verbose=2)

model.save('lenet5.1.h5')

Train on 6496 samples, validate on 1624 samples
Epoch 1/1
15s - loss: 0.1412 - val_loss: 0.0250
